In [2]:
import cv2
import torch
import urllib.request
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import gaussian_filter

Parametri intrinseci della fotocamera

In [ ]:
# Parametri intrinseci della fotocamera
fvideo = cv2.VideoCapture("./references/video/cardrive.mp4")

ret, fframe = fvideo.read()
fx = 500  # Lunghezza focale in pixel sull'asse x
fy = 500  # Lunghezza focale in pixel sull'asse y
cx = fframe.shape[1] // 2  # Centro ottico sull'asse x (al centro dell'immagine)
cy = fframe.shape[0] // 2  # Centro ottico sull'asse y (al centro dell'immagine)
fvideo.release()

Dizionario contenente la traduzione da classe a immagine (utile per la mappa 3d)

In [3]:

# Carica il modello YOLO preaddestrato (puoi sostituirlo con un modello addestrato su un tuo dataset)
yolo_model = YOLO('yolov8n.pt')  # Cambia con il tuo modello

# Carica un frame e rileva gli oggetti
frame = cv2.imread('bus.jpg')
results = yolo_model.track(frame)

# Visualizza i risultati

annotated_frame = results[0].plot()
print(results[0].boxes.id)

cv2.imshow("si", annotated_frame)
cv2.waitKey(0)

cv2.destroyAllWindows()


0: 640x480 4 persons, 1 bus, 1 stop sign, 43.5ms
Speed: 3.0ms preprocess, 43.5ms inference, 63.0ms postprocess per image at shape (1, 3, 640, 480)
tensor([1., 2., 3., 4., 5., 6.])


Istanziare il modello di stima della profondita'

In [4]:

model_type = "DPT_Large"

midas = torch.hub.load("intel-isl/MiDaS", model_type)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if model_type == "DPT_Large":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform



Using cache found in C:\Users\Edoardo/.cache\torch\hub\intel-isl_MiDaS_master
c:\Users\Edoardo\anaconda3\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Using cache found in C:\Users\Edoardo/.cache\torch\hub\intel-isl_MiDaS_master


In [5]:
print(torch.cuda.is_available())

True


In [3]:
video_path = "./references/video/cardrive.mp4"
cap = cv2.VideoCapture(video_path)

# Ottieni il framerate (FPS) del video
fps = int(cap.get(cv2.CAP_PROP_FPS))
print(fps)
frame_time = int(1000/fps)
frame_count = 0


while cap.isOpened():

    # Leggi il frame corrente
    ret, frame = cap.read()
    if not ret:
        break  # Esci se il video è terminato
    cv2.imshow('frame', frame)

    if cv2.waitKey(15) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

30


definizione della funzione midas_elaboration da inserire nel ciclo della gestione dei frame del video

In [13]:
def midas_elaboration(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    input_batch = transform(frame).to(device)
    
    # Elabora il frame (aggiungi qui il tuo codice di elaborazione)
    
    with torch.no_grad():
        prediction = midas(input_batch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=frame.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()
    output = prediction.cpu().numpy()
    output_normalized = cv2.normalize(output, None, 0, 255, cv2.NORM_MINMAX)
    output_normalized = output_normalized.astype(np.uint8)
    #cv2.imshow('frame', output_normalized)
    return output_normalized

definizione della funzione yolo_elaboration da inserire nel ciclo della gestione dei frame del video

In [14]:
def yolo_elaboration(frame):
    detection_info = []
    #carica l'immagine
    #img = cv2.imread(frame)

    #esegui il rilevamento con yolov8
    yolo_results = yolo_model.track(frame, persist=True)

    #ottieni i risultati del rilevamento (results[0] indica le informazioni ottenute dall'unica immagine in input, se ci fossero piu di una immagine allora sarebbe results[i] con i = 1...n)
    detection = yolo_results[0].boxes.xyxy #coordinate della bounding box
    confidences = yolo_results[0].boxes.conf #confidenza della rilevazione
    class_ids = yolo_results[0].boxes.cls #ID delle classi di appartenenza dell'oggetto
    obj_ids = yolo_results[0].boxes.id #ID dell'oggetto

    #visualizza i risultati
    for i, (box, conf, cls_id, obj_id) in enumerate(zip(detection, confidences, class_ids, obj_ids)):
        x1, y1, x2, y2 = map(int, box) #estrai le coordinate della bounding box

        label = f"{yolo_model.names[int(cls_id)]} ({conf:.2f})" #viene creata un'etichetta contenente il nome della classe (poiche class_ids e' un intero che grazie a model.names[cls_id] viene convertito nella corrispondente classe) e la confidenza

        detection_info.append({
            #'frame': capture.get(cv2.CAP_PROP_POS_FRAMES),
            'id': obj_id,
            'label': label,
            'confidence': conf,
            'x1': x1,
            'x2': x2,
            'y1': y1,
            'y2': y2
        })
    annotated_frame = yolo_results[0].plot()
    cv2.imshow("si", annotated_frame)
    return detection_info


0: 640x480 4 persons, 1 bus, 1 stop sign, 57.0ms
Speed: 3.0ms preprocess, 57.0ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 480)
[{'id': tensor(1.), 'label': 'bus (0.87)', 'confidence': tensor(0.8733), 'x1': 22, 'x2': 804, 'y1': 231, 'y2': 756}, {'id': tensor(2.), 'label': 'person (0.86)', 'confidence': tensor(0.8648), 'x1': 48, 'x2': 245, 'y1': 398, 'y2': 902}, {'id': tensor(3.), 'label': 'person (0.85)', 'confidence': tensor(0.8533), 'x1': 669, 'x2': 809, 'y1': 392, 'y2': 876}, {'id': tensor(4.), 'label': 'person (0.83)', 'confidence': tensor(0.8258), 'x1': 221, 'x2': 344, 'y1': 405, 'y2': 857}, {'id': tensor(5.), 'label': 'stop sign (0.26)', 'confidence': tensor(0.2606), 'x1': 0, 'x2': 32, 'y1': 254, 'y2': 324}, {'id': tensor(6.), 'label': 'person (0.25)', 'confidence': tensor(0.2506), 'x1': 0, 'x2': 63, 'y1': 550, 'y2': 873}]


combinazione dei risultati delle due funzioni precedenti per calcolare la distanza stimata degli oggetti rilevati nel frame

In [ ]:
def elaboration(midas, yolo, coefficiente, old_detection_info):
    detection_info = yolo
    depth_map = midas
    npframe = np.array(depth_map)
    updated_detection_info = []
    for object in detection_info:
        obj_id = object['id'] 
        label = object['label']
        confidence = object['confidence']
        x1 = object['x1']
        x2 = object['x2']
        y1 = object['y1']
        y2 = object['y2']
        if confidence < 0.50:
            continue
        central_x = int(x1 + x2)/2
        central_y = int(y1 + y2)/2
        """
        if any(old_obj.get('id') == obj_id for old_obj in old_detection_info):
            continue
        ##--------------------------
        """
        object_region = npframe[y1:y2, x1:x2]
        sigma = 10

        weights = gaussian_filter(np.ones_like(object_region), sigma)
        weighted_sum = gaussian_filter(object_region, sigma)
        media_pesata = np.sum(weighted_sum) / np.sum(weights)

        distanza_reale = coefficiente / media_pesata
        updated_detection_info.append({
            'id' : obj_id,
            'label' : label,
            'confidence' : confidence,
            'x' : central_x,
            'y' : central_y,
            'z' : distanza_reale
        })
    return updated_detection_info
        


definizione della funzione threedMapConversion da inserire nel ciclo della gestione dei frame del video per inserire gli oggetti rilevati in una mappa 3d

In [ ]:
def threedMapConversion(info):
    

ciclo di gestione dei frame del video

In [ ]:
video_path = "./references/video/cardrive.mp4"
cap = cv2.VideoCapture(video_path)

# Ottieni il framerate (FPS) del video
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_to_elaborate = int(fps / 4)
frame_time = int(1000/fps)
frame_count = 0
coefficiente = 100

while cap.isOpened():

    # Leggi il frame corrente
    ret, frame = cap.read()
    if not ret:
        break  # Esci se il video è terminato
    # Processa solo 1 frame ogni secondo
    if frame_count % frame_to_elaborate == 0:
        #parte MiDaS
        midas_output = midas_elaboration(frame)
        #parte YOLO
        yolo_output = yolo_elaboration(frame)
        #fu-sio-nee
        general_output = elaboration(midas_output, yolo_output, coefficiente, 4)
        """
        debug
        print(general_output)
        """
        #mappa 3D dell'ambiente circostante
        
    frame_count += 1

    if cv2.waitKey(15) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 4 cars, 5.5ms
Speed: 1.0ms preprocess, 5.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
[{'id': tensor(1.), 'label': 'car (0.72)', 'confidence': tensor(0.7190), 'real_distance': 6.23791631707584}, {'id': tensor(2.), 'label': 'car (0.71)', 'confidence': tensor(0.7112), 'real_distance': 6.666666666666667}, {'id': tensor(3.), 'label': 'car (0.57)', 'confidence': tensor(0.5664), 'real_distance': 6.666666666666667}]

0: 384x640 4 cars, 7.0ms
Speed: 0.5ms preprocess, 7.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
[{'id': tensor(1.), 'label': 'car (0.76)', 'confidence': tensor(0.7600), 'real_distance': 10.765040080940151}, {'id': tensor(2.), 'label': 'car (0.70)', 'confidence': tensor(0.7018), 'real_distance': 12.5}, {'id': tensor(3.), 'label': 'car (0.55)', 'confidence': tensor(0.5491), 'real_distance': 12.0}]

0: 384x640 5 cars, 5.5ms
Speed: 0.5ms preprocess, 5.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
[{'i

In [14]:
img = cv2.imread('dashcam.jpg')
frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
input_batch = transform(frame).to(device)
with torch.no_grad():
    prediction = midas(input_batch)

    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=frame.shape[:2],
        mode="bicubic",
        align_corners=False,
    ).squeeze()

output = prediction.cpu().numpy()

output_normalized = cv2.normalize(output, None, 0, 255, cv2.NORM_MINMAX)
output_normalized = output_normalized.astype(np.uint8)


In [21]:
cv2.imshow('a', output_normalized)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
video_path = "./references/video/highway.mp4"
cap = cv2.VideoCapture(video_path)

ret, frame = cap.read()

model = YOLO("yolov8n.pt")

tracker = Tracker()

colors = [(random.randInt(0,255), random.randInt(0,255), random.randInt(0,255)) for j in range(10)]
while ret:

    results = model(frame)

    for result in results:
        detections = []
        for r in result.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = r
            x1 = int(x1)
            x2 = int(x2)
            y1 = int(y1)
            y2 = int(y2)
            class_id = int(class_id)
            detections.append([x1, y1, x2, y2, score])
        tracker.update(frame, detections)

        for track in tracker.tracks:
            bbox = track.bbox
            x1, y1, x2, y2 = bbox
            track_id = track.track_id

            cv2.rectangle(frame, (x1, y1), (x2, y2), (colors[track_id % len(colors)]), 3)


    cv2.imshow('frame', frame)
    cv2.waitKey(30)

    ret, frame = cap.read()

cap.release()
cv2.destroyAllWindows()

In [12]:
yolo_model = YOLO("yolov8n.pt")

In [3]:
def load_midas_model():
    model_type = "DPT_Large"
    model = torch.hub.load("intel-org/MiDaS", model_type)
    model.eval()
    return model

def get_depth_map(model, image):

    image_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    transform = tv.Compose([
        tv.Resize((384, 384)),
        tv.ToTensor(),
        tv.Normalize(mean = [0.5], std = [0.5]),
    ]) 
    input_tensor = transform(image_pil).unsqueeze(0)
    with torch.no_grad():
        depth_map = model(input_tensor)
    depth_map = depth_map.squeeze().numpy()
    return depth_map

In [ ]:
midas_model = load_midas_model()

In [5]:
def detect_objects(frame):
    detection_info = []
    #carica l'immagine
    #img = cv2.imread(frame)

    #esegui il rilevamento con yolov8
    yolo_results = yolo_model(frame)

    #ottieni la mappa di profondita' con MiDaS
    depth_map = get_depth_map(midas_model, frame)

    # Ridimensiona la mappa di profondità alla risoluzione dell'immagine originale
    depth_map_resized = cv2.resize(depth_map, (frame.shape[1], frame.shape[0]))


    #ottieni i risultati del rilevamento (results[0] indica le informazioni ottenute dall'unica immagine in input, se ci fossero piu di una immagine allora sarebbe results[i] con i = 1...n)
    detection = yolo_results[0].boxes.xyxy #coordinate della bounding box
    confidences = yolo_results[0].boxes.conf #confidenza della rilevazione
    class_ids = yolo_results[0].boxes.cls #ID delle classi di appartenenza dell'oggetto

    #visualizza i risultati
    for i, (box, conf, cls_id) in enumerate(zip(detection, confidences, class_ids)):
        x1, y1, x2, y2 = map(int, box) #estrai le coordinate della bounding box

        #calcola la profondita' media nella bounding box
        depth_in_box = depth_map_resized[y1:y2, x1:x2]
        average_depth = np.mean(depth_in_box) #media della profondita'

        #label = f"{yolo_model.names[int(cls_id)]} ({conf:.2f}) - Depth: {average_depth: .2f}" #viene ncreata un'etichetta contenente il nome della classe (poiche class_ids e' un intero che grazie a model.names[cls_id] viene convertito nella corrispondente classe) e la confidenza
        label = f"Depth: {average_depth: .2f}" #viene ncreata un'etichetta contenente il nome della classe (poiche class_ids e' un intero che grazie a model.names[cls_id] viene convertito nella corrispondente classe) e la confidenza


        detection_info.append({
            #'frame': capture.get(cv2.CAP_PROP_POS_FRAMES),
            'label': label,
            'confidence': conf,
            'x1': x1,
            'x2': x2,
            'y1': y1,
            'y2': y2,
            'average_depth': average_depth
        })

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_COMPLEX, 0.9, (255, 0, 0), 2)

    #mostra l'immagine con i bounding box
    cv2.imshow("Detections", frame)
    cv2.waitKey(1)
    #cv2.destroyAllWindows()
    return detection_info

In [ ]:
video_path = "./references/video/highway.mp4"
capture = cv2.VideoCapture(video_path)
detection_info = []
while capture.isOpened():
    ret, frame = capture.read()
    if not ret:
        break #esci dal ciclo se non ci sono piu frame
    detection_info.append(detect_objects(frame))
capture.release()
cv2.destroyAllWindows()

In [ ]:
detect_objects("./references/autostrada.jpg")